In [3]:
import pandas as pd
import numpy as np
import json
import os

import DataMiner
import pymongo 

from IPython.display import display

import jellyfish as jf
from unidecode import unidecode
from statsmodels.tsa.seasonal import seasonal_decompose

In [4]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["ATFCredit"]
mycol = mydb["PureTXT"]
tables = mycol.find({})

df = pd.DataFrame()

for document in tables:
    df = df.append(document, ignore_index=True)

In [5]:
fundos_serasa = []
file_name = 'Lista de fundos na Serasa.txt'
with open(file_name) as infile:
    for line in infile: 
        aux = line.replace('\n','').strip().upper()
        if len(aux) > 0:
            fundos_serasa.append(unidecode(aux))
            
fundos_serasa = pd.Series(fundos_serasa)

FileNotFoundError: [Errno 2] No such file or directory: 'Lista de fundos na Serasa.txt'

In [6]:
df.columns

Index(['CINCO ULTIMAS CONSULTAS', 'DIVIDA VENCIDA', 'ERROR',
       'EVOLUCAO DE COMPROMISSOS - VISAO CEDENTE (VALORES EM R$)', 'FALENCIA',
       'HISTORICO DE PAGAMENTOS (QTDE DE TITULOS)', 'PEFIN', 'PENDENCIAS',
       'PROTESTO', 'REFIN', 'REGISTRO DE CONSULTAS', '_id', 'cnpj', 'txt_file',
       'HISTORICO DE PAGAMENTOS - VISAO CEDENTE',
       'HISTORICO DE PAGAMENTOS NO MERCADO (VALORES EM R$)', 'ACAO JUDICIAL'],
      dtype='object')

# 5 ultimas consultas

In [71]:
def search_factoring(name):
    values = fundos_serasa.apply(lambda x: jf.levenshtein_distance(x, name))
    result = values[values <= 4]
    
    if len(result) > 0:
        return 1
    else:
        if 'FACTORING' in name:
            return 1
        
    return 0

def count_total_factoring(df):
    return (df['IS_FACTORING'] * aux_df['QTD']).sum()

def frequency_last_five_consults(df):
    return abs(df['DATA'].diff().mean().days)

def factoring_frequency_last_five_consults(df):
    aux_df = df[df['IS_FACTORING'] >0]
    return abs(aux_df['DATA'].diff().mean().days)

In [72]:
aux_df = pd.DataFrame(df['CINCO ULTIMAS CONSULTAS'].iloc[89])
aux_df['QTD'] = aux_df['QTD'].astype(int)
aux_df['DATA'] = pd.to_datetime(aux_df['DATA'], errors = 'coerce', format = '%d/%m/%Y')

aux_df['IS_FACTORING'] = aux_df.apply(lambda x: search_factoring(x['EMPRESA']), axis = 1)
aux_df

,EMPRESA,QTD,DATA,IS_FACTORING
0,SUL AMERICA CIA NACIONAL DE SEG,1,2019-07-19,0
1,TOKIO MARINE SEGURADORA,1,2019-07-19,0
2,SULTEX IMPORTACAO E EXPORTACAO,1,2019-07-16,0
3,RED SA,1,2019-07-15,1
4,COOPERATIVA CRED RUR PLANALTO C,2,2019-07-10,1


In [73]:
print('count_total_factoring: ', count_total_factoring(aux_df))
print('frequency_last_five_consults: ', frequency_last_five_consults(aux_df))
print('factoring_frequency_last_five_consults: ', factoring_frequency_last_five_consults(aux_df))

count_total_factoring:  3
frequency_last_five_consults:  3
factoring_frequency_last_five_consults:  5


# Registro de consultas

In [129]:
def growth_trend_consults(df):
    rolling_avg = df['QTD'].rolling(window=5).mean()[-3:]
    mean = df['QTD'].mean()
    std = df[column].std()

    if len(rolling_avg[rolling_avg>= mean - std]) >= len(rolling_avg):
        return 1

    return 0

def above_average(df):
    rolling_avg = df['QTD'][-3:]
    mean = df['QTD'].mean()
    
    if len(rolling_avg[rolling_avg>= mean+0.2*mean]) > 0:
        return 1

    return 0

def total_weighted_consults(df):
    df['PESO'] = range(len(df), 0, -1)

    return (aux_df['QTD'] * aux_df['PESO']).sum()

def total_consults(df):
    return aux_df['QTD'].sum()

In [75]:
aux_df = pd.DataFrame(df['REGISTRO DE CONSULTAS'].iloc[89])
aux_df['QTD'] = aux_df['QTD'].astype(int)
aux_df

,MES,QTD
0,JUN/19,11
1,MAI/19,6
2,ABR/19,14
3,MAR/19,15
4,FEV/19,7
5,JAN/19,7
6,NOV/18,6
7,OUT/18,14
8,SET/18,12
9,AGO/18,12


In [76]:
print('growth_trend_consults: ', growth_trend_consults(aux_df))
print('above_average: ', above_average(aux_df))
print('total_weighted_consults: ', total_weighted_consults(aux_df))
print('total_consults: ', total_consults(aux_df))


growth_trend_consults:  1
above_average:  1
total_weighted_consults:  792
total_consults:  120


# PEFIN/REFIN

In [78]:
aux_df = pd.DataFrame(df['REFIN'].iloc[0])
aux_df['VALOR'] = aux_df['VALOR'].apply(lambda x: x.replace('.','')).astype(int)
aux_df['DATA'] = pd.to_datetime(aux_df['DATA'], errors = 'coerce', format = '%d/%m/%Y')


In [79]:
aux_df.iloc[0]['MODALIDADE']

'EMPRESTIMO'

In [80]:
def last_debt(df):
    return df.iloc[0]['MODALIDADE']

def most_present_debt(df):
    return df['MODALIDADE'].max()

def debt_frequency(df):
    return abs(df['DATA'].diff().mean().days)

def debt_value(df):
    return df['VALOR'].sum()

def debt_counts(df):
    return len(df)

> ## REFIN

In [81]:
print('last_debt: ', last_debt(aux_df) )
print('most_present_debt: ', most_present_debt(aux_df) )
print('debt_frequency: ', debt_frequency(aux_df) )
print('debt_value: ', debt_value(aux_df) )
print('debt_counts: ', debt_counts(aux_df) )

last_debt:  EMPRESTIMO
most_present_debt:  OUTRAS OPERA
debt_frequency:  41
debt_value:  569279
debt_counts:  4


> ## PEFIN

In [82]:
aux_df = pd.DataFrame(df['PEFIN'].iloc[0])
aux_df['VALOR'] = aux_df['VALOR'].apply(lambda x: x.replace('.','')).astype(int)
aux_df['DATA'] = pd.to_datetime(aux_df['DATA'], errors = 'coerce', format = '%d/%m/%Y')
aux_df['IS_FACTORING'] = aux_df.apply(lambda x: search_factoring(x['ORIGEM']), axis = 1)

In [83]:
def total_facdtoring_debt(df):
    return df['IS_FACTORING'].sum()

In [84]:
print('last_debt: ', last_debt(aux_df) )
print('most_present_debt: ', most_present_debt(aux_df) )
print('debt_frequency: ', debt_frequency(aux_df) )
print('debt_value: ', debt_value(aux_df) )
print('debt_counts: ', debt_counts(aux_df) )
print('total_facdtoring_debt: ', total_facdtoring_debt(aux_df) )

last_debt:  ALUGUEL
most_present_debt:  ALUGUEL
debt_frequency:  16
debt_value:  6559
debt_counts:  5
total_facdtoring_debt:  0


# Divida Vencida

In [85]:
aux_df = pd.DataFrame(df['DIVIDA VENCIDA'].iloc[0])
aux_df['VALOR'] = aux_df['VALOR'].apply(lambda x: x.replace('.','')).astype(int)
aux_df['DATA'] = pd.to_datetime(aux_df['DATA'], errors = 'coerce', format = '%d/%m/%Y')
aux_df

,DATA,MODALIDADE,VALOR,TITULO,INST_COBRADORA,LOCAL
0,2019-01-12,DEV,3818,243682913,SEFAZ RS,CAN
1,2018-12-15,DEV,89601,243664613,SEFAZ RS,CAN
2,2018-11-13,DEV,110049,243653190,SEFAZ RS,CAN
3,2018-10-23,DEV,2302,243643829,SEFAZ RS,CAN
4,2018-10-16,DEV,81132,243643632,SEFAZ RS,CAN


In [86]:
print('last_debt: ', last_debt(aux_df) )
print('most_present_debt: ', most_present_debt(aux_df) )
print('debt_frequency: ', debt_frequency(aux_df) )
print('debt_value: ', debt_value(aux_df) )
print('debt_counts: ', debt_counts(aux_df) )

last_debt:  DEV
most_present_debt:  DEV
debt_frequency:  22
debt_value:  286902
debt_counts:  5


# Historico de Pagamentos

In [144]:
def get_value(x):
    try:
        return int(x)
    except:
        try:
            x = str(x).split(' ')
            for value in x :
                try:
                    return int(value) * 1000
                except:
                    continue
            return None
        except:
            return np.nan

In [151]:
aux_df = pd.DataFrame(df['HISTORICO DE PAGAMENTOS NO MERCADO (VALORES EM R$)'].iloc[25])

aux_df['8-15_QTD'] = aux_df['8-15_QTD'].apply(get_value)
aux_df['16-30_QTD'] = aux_df['16-30_QTD'].apply(get_value)
aux_df['31-60_QTD'] = aux_df['31-60_QTD'].apply(get_value)
aux_df['+60_QTD'] = aux_df['+60_QTD'].apply(get_value)
aux_df['A_VISTA_QTD'] = aux_df['A_VISTA_QTD'].apply(get_value)


aux_df

,MES/ANO,PONTUAL_QTD,PONTUAL_%,8-15_QTD,8-15_%,16-30_QTD,16-30_%,31-60_QTD,31-60_%,+60_QTD,+60_%,A_VISTA_QTD,A_VISTA_%,TOTAL
0,DEZ/19,1.5,97.0,0,0.0,0,0.0,0,0,0,0.0,0,0,1.5
1,NOV/19,150.0,1.0,2,31.0,5,65.0,0,0,0,0.0,0,0,8.0
2,OUT/19,550.0,11.0,650,13.0,3,73.0,0,0,0,0.0,0,0,4.5
3,SET/19,550.0,31.0,1,67.0,0,0.0,0,0,0,0.0,0,0,1.5
4,AGO/19,1.0,49.0,500,23.0,500,23.0,0,0,0,0.0,0,0,2.0
5,JUL/19,2.5,55.0,1,25.0,850,17.0,0,0,0,0.0,0,0,4.5
6,JUN/19,2.0,97.0,0,0.0,0,0.0,0,0,0,0.0,0,0,2.0
7,MAI/19,1.5,61.0,1,37.0,0,0.0,0,0,0,0.0,0,0,3.0
8,ABR/19,1.5,97.0,0,0.0,0,0.0,0,0,0,0.0,0,0,1.5
9,MAR/19,1.0,97.0,0,0.0,0,0.0,0,0,0,0.0,0,0,1.0


In [127]:
def total_depts_count(df):
    return len(df)

def percent_of_debt(df, column):
    return int(df[column].count() / len(df)*100)

def total_of_debt_value(df, column):
    return df[column].sum()

def growth_trend_debt(df,column):
    rolling_avg = df[column].rolling(window=3).mean()[-3:]
    mean = df[column].mean()
    std = df[column].std()

    if len(rolling_avg[rolling_avg>= mean - std]) >= len(rolling_avg):
        return 1

    return 0
    

In [128]:
print('total_depts_count: ', total_depts_count(aux_df))
print('')

print('8-15_QTD')
print('percent_of_debt',percent_of_debt(aux_df, '8-15_QTD'))
print('total_of_debt_value',total_of_debt_value(aux_df, '8-15_QTD'))
print('growth_trend_debt',growth_trend_debt(aux_df, '8-15_QTD'))
print('')

print('16-30_QTD')
print('percent_of_debt',percent_of_debt(aux_df, '16-30_QTD'))
print('total_of_debt_value',total_of_debt_value(aux_df, '16-30_QTD'))
print('growth_trend_debt',growth_trend_debt(aux_df, '16-30_QTD'))
print('')

print('31-60_QTD')
print('percent_of_debt',percent_of_debt(aux_df, '31-60_QTD'))
print('total_of_debt_value',total_of_debt_value(aux_df, '31-60_QTD'))
print('growth_trend_debt',growth_trend_debt(aux_df, '31-60_QTD'))
print('')

print('+60_QTD')
print('percent_of_debt',percent_of_debt(aux_df, '+60_QTD'))
print('total_of_debt_value',total_of_debt_value(aux_df, '+60_QTD'))
print('growth_trend_debt',growth_trend_debt(aux_df, '+60_QTD'))
print('')

print('A_VISTA_QTD')
print('percent_of_debt',percent_of_debt(aux_df, 'A_VISTA_QTD'))
print('total_of_debt_value',total_of_debt_value(aux_df, 'A_VISTA_QTD'))
print('growth_trend_debt',growth_trend_debt(aux_df, 'A_VISTA_QTD'))

total_depts_count:  13

8-15_QTD
percent_of_debt 100
total_of_debt_value 529000
growth_trend_debt 1

16-30_QTD
percent_of_debt 100
total_of_debt_value 826000
growth_trend_debt 1

31-60_QTD
percent_of_debt 46
total_of_debt_value 28000.0
growth_trend_debt 0

+60_QTD
percent_of_debt 0
total_of_debt_value 0
growth_trend_debt 0

A_VISTA_QTD
percent_of_debt 15
total_of_debt_value 3000.0
growth_trend_debt 0


# Evolução de Compromissos

In [5]:
aux_df = pd.DataFrame(df['EVOLUCAO DE COMPROMISSOS - VISAO CEDENTE (VALORES EM R$)'].iloc[0])
aux_df

,MES/ANO,VENCIDOS,A VENCER,TOTAL
0,ABR/19,71.420,2662.960,0.0
1,MAR/19,61.388,2785.381,0.0
2,FEV/19,12.852,1928.315,0.0
3,JAN/19,187.550,908.269,0.0
4,DEZ/18,19.795,1954.514,0.0
5,NOV/18,19.234,3470.030,0.0
6,OUT/18,30.442,2796.604,0.0
7,SET/18,28.281,5604.404,0.0
8,AGO/18,34.684,6921.713,0.0
9,JUL/18,27.010,5565.894,0.0


In [6]:
aux_df['VENCIDOS']

0      71.420
1      61.388
2      12.852
3     187.550
4      19.795
5      19.234
6      30.442
7      28.281
8      34.684
9      27.010
10     44.921
11     49.886
12     11.906
13     42.756
Name: VENCIDOS, dtype: float64

In [166]:
def growth_trend_consults(df,column):
    rolling_avg = df[column].rolling(window=5).mean()[-3:]
    mean = df[column].mean()
    std = df[column].std()

    if len(rolling_avg[rolling_avg>= mean - std]) >= len(rolling_avg):
        return 1

    return 0

def total_value_commitments(df, column):
    return df[column].sum()

def count_commitments(df):
    return len(df)

In [168]:
print('count_commitments: ', count_commitments(aux_df))
print('')

print('VENCIDOS')
print('growth_trend_consults: ', growth_trend_consults(aux_df, 'VENCIDOS'))
print('total_value_commitments: ', total_value_commitments(aux_df, 'VENCIDOS'))
print('')

print('A VENCER')
print('growth_trend_consults: ', growth_trend_consults(aux_df, 'A VENCER'))
print('total_value_commitments: ', total_value_commitments(aux_df, 'A VENCER'))
print('')

print('TOTAL')
print('growth_trend_consults: ', growth_trend_consults(aux_df, 'TOTAL'))
print('total_value_commitments: ', total_value_commitments(aux_df, 'TOTAL'))



count_commitments:  14

VENCIDOS
growth_trend_consults:  1
total_value_commitments:  642.125

A VENCER
growth_trend_consults:  1
total_value_commitments:  51515.693

TOTAL
growth_trend_consults:  1
total_value_commitments:  0.0


# Falencia

In [8]:
df.iloc[0]

CINCO ULTIMAS CONSULTAS                                     {'EMPRESA': {'0': 'NOVA AMERICA FACTORING LTDA...
DIVIDA VENCIDA                                              {'DATA': {'0': '12/01/2019', '1': '15/12/2018'...
ERROR                                                       [HISTORICO DE PAGAMENTOS - VISAO CEDENTE, HIST...
EVOLUCAO DE COMPROMISSOS - VISAO CEDENTE (VALORES EM R$)    {'MES/ANO': {'0': 'ABR/19', '1': 'MAR/19', '2'...
FALENCIA                                                    {'DATA': {'0': '17/06/2014'}, 'TIPO': {'0': 'R...
HISTORICO DE PAGAMENTOS (QTDE DE TITULOS)                   {'PONTUAL_QTD': {'0': 705.0}, 'PONTUAL_%': {'0...
PEFIN                                                       {'DATA': {'0': '28/06/2017', '1': '28/06/2017'...
PENDENCIAS                                                  {'TOTAL DE OCORRENCIAS': {'0': '4'}, 'VALOR TO...
PROTESTO                                                    {'DATA': {'0': '14/07/2018', '1': '17/04/2018'...
REFIN     

In [7]:
aux_df = pd.DataFrame(df['FALENCIA'].iloc[0])
aux_df

,DATA,TIPO,ORIGEM,CIDADE,UF
0,17/06/2014,RECUPERAC JUDIC REQ,VARA02,CURITIBA,PR


In [185]:
    def bankruptcy_type_count(df, tipo = 'REQ'):
        df[tipo] = 0
        df.loc[df['TIPO'].str.contains(tipo), tipo] = 1

        return df[tipo].sum()

In [186]:
print('bankruptcy_type_count_REQ: ', bankruptcy_type_count(aux_df, 'REQ'))
print('bankruptcy_type_count_CONC: ', bankruptcy_type_count(aux_df, 'CONC'))

bankruptcy_type_count REQ:  1
bankruptcy_type_count CONC:  0


# Acções Judiciais

In [209]:
for i in range(len(df)):
    try:
        print(df['txt_file'].iloc[i], ' -> ', i)
        display(pd.DataFrame(df['PROTESTO'].iloc[i]))
    except:
        pass

SERASA-R1059038.txt  ->  0


,DATA,VALOR,CARTORIO,CIDADE,UF
0,14/07/2018,125.782,01,CANOAS,RS
1,17/04/2018,136.324,01,CANOAS,RS
2,15/02/2018,119.655,01,CANOAS,RS
3,13/01/2018,124.577,01,CANOAS,RS
4,14/11/2017,141.195,01,CANOAS,RS


SERASA-R1084436.txt  ->  1
SERASA-R1109513.txt  ->  2


,DATA,VALOR,CARTORIO,CIDADE,UF
0,14/06/2013,2.566,UN,ARAPONGAS,PR
1,14/06/2013,36.492,UN,ARAPONGAS,PR
2,08/02/2013,70.610,04,CUIABA,MT
3,04/02/2013,18.585,UN,APUCARANA,PR
4,04/02/2013,20.029,UN,ARAPONGAS,PR


SERASA-R1146803.txt  ->  3


...

SERASA-R798713.txt  ->  86


,DATA,VALOR,CARTORIO,CIDADE,UF
0,16/09/2010,2.446,UN,ELDORADO DO SUL,RS
1,16/09/2010,3.316,UN,ELDORADO DO SUL,RS
2,16/09/2010,1.384,UN,ELDORADO DO SUL,RS
3,16/09/2010,3.510,UN,ELDORADO DO SUL,RS
4,16/09/2010,275,UN,ELDORADO DO SUL,RS


SERASA-R855701.txt  ->  87
SERASA-R873591.txt  ->  88


,DATA,VALOR,CARTORIO,CIDADE,UF
0,21/12/2018,7.913,UN,SAO JOAO BATISTA,SC
1,17/12/2018,4.274,UN,SAO JOAO BATISTA,SC
2,10/07/2017,143.623,UN,SAO JOAO BATISTA,SC


SERASA-R907294.txt  ->  89


In [9]:
df.iloc[5]

CINCO ULTIMAS CONSULTAS                                     {'EMPRESA': {'0': 'BENFICA FD INVEST DIR CREDI...
DIVIDA VENCIDA                                                                                            NaN
ERROR                                                       [HISTORICO DE PAGAMENTOS - VISAO CEDENTE, HIST...
EVOLUCAO DE COMPROMISSOS - VISAO CEDENTE (VALORES EM R$)    {'MES/ANO': {'0': 'OUT/18', '1': 'SET/18', '2'...
FALENCIA                                                                                                  NaN
HISTORICO DE PAGAMENTOS (QTDE DE TITULOS)                   {'PONTUAL_QTD': {'0': 189.0}, 'PONTUAL_%': {'0...
PEFIN                                                                                                     NaN
PENDENCIAS                                                                                                NaN
PROTESTO                                                    {'DATA': {'0': '08/08/2018'}, 'VALOR': {'0': '...
REFIN     

In [205]:
aux_df = pd.DataFrame(df['ACAO JUDICIAL'].iloc[5])
aux_df['DATA'] = pd.to_datetime(aux_df['DATA'], errors = 'coerce', format = '%d/%m/%Y')
aux_df

,DATA,NATUREZA,AVAL,VALOR,DIST,VARA,CIDADE,UF
0,2017-10-14,EXECUCAO,,383.406,01,01,PORTO ALEGRE,RS


In [206]:
def most_present_lawsuit(df):
    return df['NATUREZA'].max()

def total_lawsuit(df):
    return len(df)

def total_value(df):
    try:
        df['VALOR'] = df['VALOR'].apply(lambda x: x.replace('.',''))
        return df['VALOR'].astype(int).sum()
    except:
        return 0
    
def frequency_lawsuit(df):
    return abs(df['DATA'].diff().mean().days)

In [207]:
print('most_present_lawsuit: ', most_present_lawsuit(aux_df))
print('total_lawsuit: ', total_lawsuit(aux_df))
print('total_value: ', total_value(aux_df))
print('frequency_lawsuit: ', frequency_lawsuit(aux_df))

most_present_lawsuit:  EXECUCAO
total_lawsuit:  1
total_value:  383406
frequency_lawsuit:  nan


# Protesto

In [208]:
df.columns

Index(['CINCO ULTIMAS CONSULTAS', 'DIVIDA VENCIDA', 'ERROR',
       'EVOLUCAO DE COMPROMISSOS - VISAO CEDENTE (VALORES EM R$)', 'FALENCIA',
       'HISTORICO DE PAGAMENTOS (QTDE DE TITULOS)', 'PEFIN', 'PENDENCIAS',
       'PROTESTO', 'REFIN', 'REGISTRO DE CONSULTAS', '_id', 'cnpj', 'txt_file',
       'HISTORICO DE PAGAMENTOS - VISAO CEDENTE',
       'HISTORICO DE PAGAMENTOS NO MERCADO (VALORES EM R$)', 'ACAO JUDICIAL'],
      dtype='object')

In [213]:
aux_df = pd.DataFrame(df['PROTESTO'].iloc[28])
aux_df['DATA'] = pd.to_datetime(aux_df['DATA'], errors = 'coerce', format = '%d/%m/%Y')
aux_df['VALOR'] = aux_df['VALOR'].apply(lambda x: x.replace('.',''))
aux_df['VALOR'] = aux_df['VALOR'].astype(int)
aux_df

,DATA,VALOR,CARTORIO,CIDADE,UF
0,2019-10-10,469,02,LONDRINA,PR
1,2019-10-09,11300,02,LONDRINA,PR
2,2019-09-27,74,03,LONDRINA,PR
3,2019-09-22,637,03,LONDRINA,PR
4,2019-09-12,79,01,LONDRINA,PR


In [218]:
def total_protest(df):
    return len(df)
    
def std_protest(df):
    return df['VALOR'].std()
    
def mean_protest(df):
    return df['VALOR'].mean()

def frequency_protest(df):
    return abs(df['DATA'].diff().mean().days)

In [219]:
print('total_protest: ', total_protest(aux_df))
print('std_protest: ', std_protest(aux_df))
print('mean_protest: ', mean_protest(aux_df))
print('frequency_protest: ', frequency_protest(aux_df))

total_protest:  5
std_protest:  4918.885818963478
mean_protest:  2511.8
frequency_protest:  7
